In [1]:
import numpy as np
import cv2
import hashlib
import pandas as pd

In [20]:
# CONSTANTS
IMAGE = cv2.imread("lava_lamp.jpeg")
THRESHOLD = 0.25
RESIZED_IMG = cv2.cvtColor(cv2.resize(IMAGE, (600, 600)), cv2.COLOR_BGR2RGB)
EXP_RES_IMG = RESIZED_IMG.copy()

In [2]:
def get_available_cameras():
    available_cameras = []
    # Check for 5 cameras
    for i in range(5):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()
    return available_cameras


cameras = get_available_cameras()
if cameras:
    print("Available Cameras:", cameras)
else:
    print("No cameras found.")

Available Cameras: [0, 1]


In [11]:
def create_combined_window(frames_dict, target_width=800, target_height=None):
    first_frame = next(iter(frames_dict.values()))
    height, width = first_frame.shape[:2]

    # Dimensions maintaining aspect ratio
    target_height = int(target_width * height / width) if target_height is None else target_height

    # Blank canvas to hold all frames
    combined = np.zeros(
        (target_height, target_width * len(frames_dict), 3), dtype=np.uint8
    )

    # Add each frame to  canvas
    for idx, (name, frame) in enumerate(frames_dict.items()):
        # Resize frame
        resized_frame = cv2.resize(frame, (target_width, target_height))

        # Grayscale -> BGR (if ever)
        if len(resized_frame.shape) == 2:
            resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2BGR)

        # Frame -> Canvas
        combined[:, idx * target_width : (idx + 1) * target_width] = resized_frame

        # Labeling on top left
        cv2.putText(
            combined,
            name,
            (idx * target_width + 10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 255, 255),
            2,
        )

    return combined


def get_hash(x, y, w, h, cx, cy):
    data = f"{x}, {y}, {w}, {h}, {cx}, {cy}"
    d_bytes = data.encode("utf-8")
    d_hash = hashlib.sha512(d_bytes).hexdigest()
    return d_hash

In [ ]:
df = []
cap = cv2.VideoCapture(1)
while True:
    mean_data = {"x": [], "y": [], "w": [], "h": [], "cx": [], "cy": []}
    ret, frame = cap.read()
    if not ret:
        print("Failed!")
        break

    gr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    conv_frame = cv2.threshold(gr, 210, 255, cv2.THRESH_BINARY)[1]
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
        conv_frame, 12, cv2.CV_32S
    )

    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cx, cy = int(centroids[i][0]), int(centroids[i][1])
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

        mean_data["x"].append(x)
        mean_data["y"].append(y)
        mean_data["w"].append(w)
        mean_data["h"].append(h)
        mean_data["cx"].append(cx)
        mean_data["cy"].append(cy)

    combined = create_combined_window(
        {"Original": frame, "Binary": conv_frame, "Gray": gr},
        target_width=480, 
        target_height=360
    )

    mean_frame = pd.DataFrame(mean_data)
    hash = get_hash(*mean_frame.mean().values)
    df.append(mean_frame.mean().tolist())
    df[-1].append(hash)
    
    # Single window with both frames
    cv2.imshow("Lava Lamp Detection", combined)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
display(pd.DataFrame(df, columns=mean_frame.columns.tolist() + ["hash"]))

In [16]:
df = pd.DataFrame(df, columns=mean_frame.columns.tolist() + ["hash"])
df.to_csv("lava_lamp.csv", index=False)